## Data Wrangling (Gathering, Accessing, and Cleaning)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import sys

In [2]:
distances_df = pd.read_csv('https://raw.githubusercontent.com/Haidarr-h/datasetOptimasi1/main/distance.csv')

In [3]:
distances_df.head()

,UserId,4,38,71,90,94,101,142,163,164,...,8206,8232,8236,8255,8271,8300,8358,8360,8507,8545
0,4,0,1280,1762,1406,1589,1312,941,1098,1020,...,1377,321,1469,1398,1753,1762,846,777,1025,1017
1,38,1280,0,681,202,385,956,1639,1003,1520,...,1817,1178,411,2029,1370,681,1603,1408,1515,1433
2,71,1762,681,0,503,338,1512,2110,1524,2002,...,2323,1660,342,2511,1926,0,2084,1890,1996,1915
3,90,1406,202,503,0,208,1082,1765,1129,1646,...,1943,1304,355,2155,1496,503,1728,1534,1640,1559
4,94,1589,385,338,208,0,1264,1948,1311,1829,...,2126,1487,226,2338,1679,338,1911,1717,1823,1742


## Genetic Algorithm

#### Graph Visit Locations

In [96]:
locations_connections = {}
location_set = set()

def exist_connection(source, destination):
    return (True if (source, destination) in locations_connections else False)

def add_connection(source, destination, cost=0):
    if not exist_connection(source, destination):
        locations_connections[(source, destination)] = cost
        location_set.add(source)
        location_set.add(destination)

def get_cost_path(path):
    total_cost = 0
    for i in range(len(path)-1):
        if (path[i], path[i+1]) in locations_connections:
            total_cost = total_cost + locations_connections[(path[i], path[i+1])]
        else:
            print("Cost Antar Lokasi Tidak Ditemukan")
    return total_cost

def get_random_path(max_population):
    random_path = []
    location_list = list(location_set)
    first_location = random.choice(location_list)
    
    if first_location not in location_set:
        print("Lokasi Awal Pembentukan Path Tidak Ditemukan")
        sys.exit(1)
    
    location_list.remove(first_location)
    location_list.insert(0, first_location)

    for i in range(max_population):
        location_list_temp = location_list[1:]
        random.shuffle(location_list_temp)
        location_list_temp.insert(0, first_location)

        if location_list_temp not in random_path:
            random_path.append(location_list_temp)
    
    return random_path

def show_connections():
    print("Showing Locations Connections...")

    for location in locations_connections:
        print("Source :", location[0], "| Destination :", location[1], "| Cost", locations_connections[location])

#### Selection, Crossover, and Mutation Function

In [97]:
def select_parent(population, chromosome_amount):
    tournament_participants = random.choices(population, k=chromosome_amount)
    fitness_cost = [get_cost_path(chromosome) for chromosome in tournament_participants]
    best_chromosome_index = fitness_cost.index(min(fitness_cost))
    selected_parent = tournament_participants[best_chromosome_index]

    return selected_parent

def check_identical_parents(parent_1, parent_2):
    if len(parent_1)!=len(parent_2):
        print("Parents have different leng of chromosomes. Please kindly check it !")
        sys.exit(1)

    for i in range(len(parent_1)):
        if parent_1[i]!=parent_2[i]:
            return False

    return True

def crossover(parent_1, parent_2, point):
    child_chromosome = parent_1[0:point]

    for i in parent_2:
        if i not in child_chromosome:
            child_chromosome.append(i)

    return child_chromosome

def mutation(child, location_counts):
    index_1 = random.randint(0, location_counts-1)
    index_2 = random.randint(0, location_counts-1)

    while index_1 == index_2:
        index_2 = random.randint(0, location_counts-1)

    child[index_1], child[index_2] = child[index_2], child[index_1]

    return child

#### Genetic Algorithm Run

In [98]:
def run_genetic_algorithm(population_size, iteration, location_counts, tournament_selection_rate, mutation_rate, crossover_rate):
    generation_number = 0
    population = []
    population_cost = [0]*population_size
    
    # creating the first ever population
    population = get_random_path(population_size)
    # calculating the fitness of each chromosome
    for i in range(population_size):
        population_cost[i] = get_cost_path(population[i])

    # check the first population
    if not population:
        print("There are no population has been initialized yet, please kindly check the code!")
        sys.exit(1)
    
    # Main Loop
    for i in range(iteration):
        # Sorted the cost based on the index
        sorted_index_score = np.argsort(population_cost)
        new_population = []
        #new_population_cost = [0]*population_size

        # Pick the best two of the populations (elitism)
        new_population.append(population[sorted_index_score[0]])
        new_population.append(population[sorted_index_score[1]])

        for i in range((population_size-2)//2):
            # Crossovers
            if random.random() < crossover_rate:
                parent_1 = select_parent(population, tournament_selection_rate)
                parent_2 = select_parent(population, tournament_selection_rate)

                while(check_identical_parents(parent_1, parent_2) == True):
                    parent_2 = select_parent(population, tournament_selection_rate)

                point = random.randint(0, location_counts-1)
                child_1 = crossover(parent_1, parent_2, point)
                child_2 = crossover(parent_1, parent_2, point)
                
            # If crossover not happen
            else :
                child_1 = random.choices(population)[0]
                child_2 = random.choices(population)[0]

            # Mutation
            if random.random() < mutation_rate:
                child_1 = mutation(child_1, location_counts)
                child_2 = mutation(child_2, location_counts)


            new_population.append(child_1)
            new_population.append(child_2)

        population = new_population
        for i in range(population_size):
            population_cost[i] = get_cost_path(population[i])

        generation_number += 1

        best_chromosome = population[population_cost.index(min(population_cost))]

        if get_cost_path(best_chromosome) != min(population_cost):
            print("There is a difference between best Chromosome and Minimum Cost")
        
        if generation_number % 10 == 0:
            print('Generation :', generation_number, 'Best Chromosome Cost:', get_cost_path(best_chromosome))

    print('The Shortest Cost is', get_cost_path(best_chromosome))
    print('The shortest Route :', best_chromosome)
    print('Total Gen Number :', generation_number)
    

#### Main

In [101]:
def main():
    for i in distances_df.index:
      for x, j in enumerate(distances_df.columns[1:], start=1):
        j = int(j)
        # add source, destination, dan duration cost dari dataframe
        add_connection(distances_df.iloc[i,0], j, distances_df.iloc[i, x])
    
    population_size = 300
    iterations = 3000
    location_counts = len(location_set)
    tournament_selection_rate = 4
    mutation_rate = 0.1
    crossover_rate = 0.95

    #show_connections()

    run_genetic_algorithm(
        population_size, iterations, location_counts, tournament_selection_rate, mutation_rate, crossover_rate
    )

main()

236
Generation : 10 Best Chromosome Cost: 253280
Generation : 20 Best Chromosome Cost: 240790
Generation : 30 Best Chromosome Cost: 226712
Generation : 40 Best Chromosome Cost: 212834
Generation : 50 Best Chromosome Cost: 201913
Generation : 60 Best Chromosome Cost: 194029
Generation : 70 Best Chromosome Cost: 186543
Generation : 80 Best Chromosome Cost: 181414
Generation : 90 Best Chromosome Cost: 179496
Generation : 100 Best Chromosome Cost: 176494
Generation : 110 Best Chromosome Cost: 171221
Generation : 120 Best Chromosome Cost: 167945
Generation : 130 Best Chromosome Cost: 165926
Generation : 140 Best Chromosome Cost: 164197
Generation : 150 Best Chromosome Cost: 162288
Generation : 160 Best Chromosome Cost: 157165
Generation : 170 Best Chromosome Cost: 156026
Generation : 180 Best Chromosome Cost: 152611
Generation : 190 Best Chromosome Cost: 150226
Generation : 200 Best Chromosome Cost: 148419
Generation : 210 Best Chromosome Cost: 147295
Generation : 220 Best Chromosome Cost: 

In [88]:
x = [6857, 5259, 280, 2698, 196, 3509, 1508, 7514, 1175, 6819, 6856, 4661, 2442, 4531, 7020, 6451, 3234, 304, 1116, 2670, 101, 5545, 7599, 6972, 5023, 4669, 3648, 164, 8507, 213, 7578, 3052, 1560, 7028, 5264, 6966, 2680, 142, 5068, 985, 8545, 38, 7573, 554, 8201, 163, 5223, 7289, 1363, 5143, 3839, 4655, 5706, 5474, 5639, 8206, 3214, 6405, 4398, 7352, 6048, 3120, 4484, 216, 520, 4356, 1407, 760, 2736, 4839, 1076, 4027, 4120, 1337, 4794, 4535, 2056, 94, 4525, 5665, 1848, 1626, 4703, 1073, 5757, 7158, 7304, 8271, 7512, 5655, 8255, 568, 3241, 5227, 5630, 1138, 1989, 7068, 2451, 8232, 3643, 6540, 1824, 4859, 683, 4603, 3408, 464, 6898, 7396, 1304, 7649, 2091, 3647, 1788, 1907, 7320, 3436, 5500, 7328, 5958, 3518, 6983, 2717, 4150, 3801, 2619, 8172, 8358, 653, 8300, 6331, 336, 8360, 6322, 2713, 3764, 7270, 1767, 4936, 2354, 7629, 5669, 1758, 466, 3091, 1918, 201, 1467, 5408, 1808, 6796, 6007, 1468, 7567, 3118, 4322, 1973, 8177, 1388, 3218, 5308, 671, 3897, 6993, 1540, 3348, 5950, 6078, 7108, 6658, 3359, 3862, 1327, 1926, 940, 1631, 8195, 4100, 90, 6846, 5261, 7346, 1430, 3040, 4300, 4357, 3451, 2139, 3507, 6967, 1231, 918, 864, 1039, 8236, 2192, 4966, 3606, 3719, 1310, 1538, 664, 5520, 2265, 2176, 4758, 3242, 5939, 7591, 7120, 2384, 5492, 7549, 7372, 4, 8179, 71, 4570, 2002, 3828, 7071, 6432, 5501, 1136, 3268, 6454, 6891, 8143, 607, 2048, 1071, 5895, 5415, 1412, 3431]
get_cost_path(x)
#print(all(element in x for element in location_set))
#print(all(element in location_set for element in x))

91286